# Wykład 5 - Generyki

## 5.1 Generyczne argumenty typowane

In [10]:
val lecturers = listOf("Rafał", "Robert") // domniemanie argumentów typowanych
val lecturersList: List<String> = listOf("Rafał", "Robert")

In [8]:
List<String> lecturers = new ArrayList<String>();
lecturers.add("Rafał");
lecturers.add("Robert");

List<String> list = Arrays.asList("Rafał", "Robert");
List list = Arrays.asList("Rafał", "Robert"); // typ surowy

## 5.2 Funkcje gerenyczne

Jeżeli potrzebujemy funkcję przetwarzającą dowolną listę, a nie listę elementów o określonym typie, możemy wykorzystać funkcję generyczną

In [37]:
fun <T> List<T>.destruct(ind: Int):Pair<List<T>, List<T>> = Pair(take(ind), drop(ind))

- `fun<T>` - deklaracja argumentu typowanego
- `List<T>` - typ odbiornika funkcji
- `Pair<List<T>, List<T>>` - typ zwracanego wyniku

In [38]:
val lista = listOf(1, 2, 3, 4, 5)
println(lista.destruct(1))

([1], [2, 3, 4, 5])


In [39]:
val listb = listOf('a', 'b', 'c', 'd', 'e')
println(listb.destruct(1))

([a], [b, c, d, e])


In [46]:
val <T> List<T>.przedostatni: T
    get() = this[size - 2]

println(listOf(1, 2, 3, 4, 5).przedostatni)
println(listOf("Rafał", "Robert", "Ania").przedostatni)

4
Robert


## 5.3 Klasy generyczne

In [53]:
class Store<T>(t: T) {
    var a = t
}

val store: Store<Int> = Store<Int>(1)
println(store.a)

1


In [54]:
val store2 = Store(1)
println(store2.a)

1


In [55]:
val store3 = Store("Rafał")
println(store3.a)

Rafał


## 5.4 Interfejsy generyczne

In [68]:
interface Porownaj<T> {
    fun porownajDo(other: T): String
}

class Student(name: String): Porownaj<Student> {
    val name: String = name
    
    override fun porownajDo(other: Student): String = 
        if (this.name.first() > other.name.first())
            this.name else other.name
}

In [72]:
val rafal = Student("Rafał")
val ania = Student("Ania")

println(rafal.porownajDo(ania))

Rafał


In [73]:
println(ania.porownajDo(rafal))

Rafał


## 5.5 Granice

Jeżeli chcemy ograniczyć listę typów, przykładowo chcemy obliczyć sumę, możemy zastosować granice typów.

In [76]:
fun <T: Number> half(value: T): Double {
    return value.toDouble() / 2.0
}

println(half(3.0))

1.5


- klasa `Number` (superklasa wszyskich klas reprezentujących wartości liczbowe) jest **górną** granicą argumentu typowanego

In [77]:
println(half("a"))

Line_76.jupyter-kts (1:14 - 17) Type mismatch: inferred type is String but Number was expected

In [78]:
fun <T: Comparable<T>> max (first: T, second: T): T {
    return if (first > second) first else second
}

println(max("Rafał", "Ania"))

Rafał


In [79]:
println(max(1, 2))

2


In [82]:
class Student(name: String): Comparable<Student> {
    val name: String = name
    
    override fun compareTo(other: Student): Int = 
        if (this.name.first() > other.name.first())
            1 else 2
}

In [84]:
val rafal = Student("Rafał")
val ania = Student("Ania")

println(rafal.compareTo(ania))

1


In [88]:
fun <T> ensureTrailingPeriod(seq: T)
    where T: CharSequence, T: Appendable {
    
    if (!seq.endsWith('.'))
        seq.append('.')
}
    
val hello = StringBuilder("Hello WpumKJ")
ensureTrailingPeriod(hello)
println(hello)

Hello WpumKJ.


## 5.6 Klasy kowariantne, kontrawariantne i inwariantne

Mamy klasę `X<T>`. Jeżeli `A` jest podtypem `B`, czyli `B: A`, to `X<A>` jest podtypem `X<B>`

Innymi słowy, klasa kowariantna to klasa generyczna zachowująca zależność między podtypami

In [116]:
open class University
open class Department : University()
class ISSP : Department()

In [117]:
class Lectures<out T> {
    private val contents = mutableListOf<T>()
    fun produce(): T = contents.last()
    //fun consume(item: T) = contents.add(item)
}

In [118]:
fun useProducer(l: Lectures<Department>) {
    val lecture = l.produce()
}

In [124]:
//useProducer(Lectures<ISSP>())
//useProducer(Lectures<Department>())
//useProducer(Lectures<University>())

Line_123.jupyter-kts (3:13 - 35) Type mismatch: inferred type is Line_116.Lectures<Line_115.University> but Line_116.Lectures<Line_115.Department> was expected

In [135]:
class Lectures<in T> {
    private val contents = mutableListOf<T>()
    //fun produce(): T = contents.last()
    fun consume(item: T) = contents.add(item)
}

In [139]:
fun useConsumer(l: Lectures<Department>) {
    l.consume(ISSP())
}

`open class A`
`class B: A`

- klasa kowariantna `Class<out T>` - zależność pomiędzy podtypami jest zachowana (`class<B> jest podtypem class<A>`), typ `T` można stosować tylko w pozycji wyjściowej
- klasa kontrawariantna `Class<in T>` - zależność pomiędzy podtypami jest odwrócona (`class<A>` jest podtypem `class<B>`), typ `T` można stosować tylko w pozycji wejściowej
- klasa inwariantna `Class<T>` - brak zależności pomiędzy podtypami, typ `T` może być w dowolnej pozycji

## 5.7 Projekcja `*` (typ projektowany)

Typ `MutableList<Any?>` może zawierać obiekty **dowolnego** typu<br>
Typ `MutableList<*>` może zawierać obiekty **jednego** typu, ale nie wiadomo jakiego

In [143]:
val l: MutableList<Any?> = mutableListOf('a', 1, "Rafał", null)
val star: MutableList<*> = mutableListOf('a', 'b', 'c')

//stars.add(4)